In [6]:
cd ../src

/home/marc/gal/repos/supervised-learning-case-study/interview/src


In [30]:
from main import combine_types, convert_to_boolean, clean_col, create_local_col, df_col_setup

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [32]:
cols = np.arange(0,23)
df = pd.read_csv("../data/Interview.csv", usecols=cols)
df.drop(1233, inplace=True)
df.drop(['Candidate Current Location', 'Name(Cand ID)'], axis=1, inplace=True)

In [33]:
col_rename_dict = {
        'Date of Interview': 'Date',
        'Client name': 'Company',
        'Position to be closed': 'Position',
        'Nature of Skillset': 'Skillset',
        "Have you obtained the necessary permission to start at the required time": "Permissions",
        'Hope there will be no unscheduled meetings': 'No Unscheduled Meetings',
        "Can I Call you three hours before the interview and follow up on your attendance for the interview": '3 Hour Confirmation Call',
        "Can I have an alternative number/ desk number. I assure you that I will not trouble you too much": "Alternate Phone Number",
        'Have you taken a printout of your updated resume. Have you read the JD and understood the same': "Took Resume and Read JD",
        'Are you clear with the venue details and the landmark.' : 'Confirmed Location',
        'Has the call letter been shared' : 'Call Letter Shared',
        'Marital Status' : 'Married'
    }

df.rename(columns = col_rename_dict, inplace = True)
cols_with_NaN = ['No Unscheduled Meetings', 'Permissions',
                 'Alternate Phone Number', 'Took Resume and Read JD', 'Confirmed Location', 'Call Letter Shared', '3 Hour Confirmation Call', 'Expected Attendance']
for col in cols_with_NaN:
    df[col].fillna('Na', inplace = True)

clean_col(df, ["Observed Attendance", 'No Unscheduled Meetings', "Candidate Job Location", 
                "Location", 'Permissions', 'Alternate Phone Number', 'Call Letter Shared', '3 Hour Confirmation Call', 'Took Resume and Read JD',
                'Confirmed Location'])

# If Not sure or NA is provided, grouped into No for boolean values - may reconsider adding third "Unknown" entry later
cols_to_bool = [('Observed Attendance', 'yes'),
                ('Married', 'Married'), 
                ('Gender', 'Male'),
                ('No Unscheduled Meetings', 'yes'),
                ('Permissions', 'yes'),
                ('Alternate Phone Number', 'yes'),
                ('3 Hour Confirmation Call', 'yes'),
                ('Call Letter Shared', 'yes'),
                ('Took Resume and Read JD', 'yes'),
                ('Confirmed Location', 'yes')
                ]
for col, def_1 in cols_to_bool:
    convert_to_boolean(df, col, def_1)


#Combine redundant IT variations for Company into IT
combine_types(df, col='Industry', other_entries=('IT Services', 'IT Products and Services'), combine_to = 'IT')
combine_types(df, col='Location', other_entries=(
    'gurgaonr'), combine_to='gurgaon')
combine_types(df, 'Expected Attendance', ['NO', 'Uncertain', 'Na'], 'No')
combine_types(df, 'Expected Attendance', ['yes', '10.30 Am', '11:00 AM'], 'Yes')
create_local_col(df)

In [ ]:
for i, exp_attend in enumerate(df['Expected Attendance']):
    if exp_attend == 'Yes':
        df.iloc[i, df.columns.get_loc('Expected Attendance')] = 1
    else:
        df.iloc[i, df.columns.get_loc('Expected Attendance')] = 0

In [36]:
df.head()

,Date,Company,Industry,Location,Position,Skillset,Interview Type,Gender,Candidate Job Location,Interview Venue,...,No Unscheduled Meetings,3 Hour Confirmation Call,Alternate Phone Number,Took Resume and Read JD,Confirmed Location,Call Letter Shared,Expected Attendance,Observed Attendance,Married,Local Candidate
0,13.02.2015,Hospira,Pharmaceuticals,chennai,Production- Sterile,Routine,Scheduled Walkin,1,hosur,Hosur,...,1,1,1,1,1,1,1,0,0,0
1,13.02.2015,Hospira,Pharmaceuticals,chennai,Production- Sterile,Routine,Scheduled Walkin,1,bangalore,Hosur,...,1,1,1,1,1,1,1,0,0,0
2,13.02.2015,Hospira,Pharmaceuticals,chennai,Production- Sterile,Routine,Scheduled Walkin,1,chennai,Hosur,...,0,0,0,0,0,0,0,0,0,1
3,13.02.2015,Hospira,Pharmaceuticals,chennai,Production- Sterile,Routine,Scheduled Walkin,1,chennai,Hosur,...,1,0,1,0,1,1,0,0,0,1
4,13.02.2015,Hospira,Pharmaceuticals,chennai,Production- Sterile,Routine,Scheduled Walkin,1,bangalore,Hosur,...,1,1,0,1,1,1,0,0,1,0


In [38]:
their_acc_sum = []
for their_guess, obs_attend in zip(df['Expected Attendance'],df['Observed Attendance']):
    if their_guess == obs_attend:
        their_acc_sum.append(1)
    else:
        their_acc_sum.append(0)
their_acc = np.average(their_acc_sum)
their_acc

0.708029197080292